In [5]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import ast
import torch
import os

In [20]:
from src.util_func import load_yaml,z_score_normalize,get_top_k_indices,run_command,round_down_to_power_of_two,load_yaml_args,normalize_to_neg1_pos1


### Vary Selection Size

### Main Method MELD-DS

In [6]:
def z_score_normalize(sample_IF):
    """
    对sample_IF字典中各个'method'对应的值（键为index、值为float的字典）进行Z-score归一化。

    参数:
    sample_IF (dict): 包含多个'method'键的字典，每个'method'键对应的值为需要进行归一化处理的字典数据。

    返回:
    dict: 归一化后的字典，结构与输入的sample_IF一致，其中每个'method'键对应的值都已经完成Z-score归一化。
    """
    for method in sample_IF.keys():
        # 获取当前method对应需要归一化的值列表，保持原有顺序
        values_list = list(sample_IF[method].values())
        # 将列表转换为torch.Tensor
        tensor_value = torch.tensor(values_list).unsqueeze(1)  # 添加维度，变为二维张量

        # 计算均值和标准差
        mean_value = tensor_value.mean()
        std_value = tensor_value.std()

        # 进行Z-score归一化
        normalized_tensor = (tensor_value - mean_value) / std_value

        # 将归一化后的结果再转换回列表
        normalized_list = normalized_tensor.squeeze(1).tolist()

        # 更新原字典中当前method对应的值
        index_list = list(sample_IF[method].keys())
        normalized_dict = {}
        for index, normalized_value in zip(index_list, normalized_list):
            normalized_dict[index] = normalized_value

        sample_IF[method] = normalized_dict

    return sample_IF

In [19]:
## reverse total score
import json
import numpy as np
import pandas as pd
import torch
from src.util_func import load_yaml
task_list = [
    # ['RE','RE'],
    # ['ER','abt-buy'],
    # ['ER','semi-text-w'],
    # ['DC','hospital'],
    # ['DC','beer'],
    # ['DC','rayyan'],
    # ['DI','amazon'],
    # ['DI','walmart'],
    # ['AVE','oa_mine'],
    # ['CTA','SimTab'],
    # ['CTA','WebTable'],
    # ['ER','walmart-amazon'],
    # ['ER','semi-text-c'],
    ['ER','amazon-google'],
    # ['ER','wdc'],
    # ['SM','CMS']
]
for task,dataset in task_list:
    yaml_path = f'script/config_{task}_{dataset}.yaml'
    config = load_yaml(yaml_path)
    cluster_num = config['cluster_num']
    cluster_per_budget = config['sample_per_cluster']
    cluster_per_budget = 3
    train_file_path = config['train_file_path']
    train_file = pd.read_json(train_file_path)

    cluster_rank = torch.load(f'../DataSelection-IF/selection/{task}/{dataset}/Cluster-Rank.pkl',weights_only=False)
    total_score = torch.load(f'../DataSelection-IF/selection/{task}/{dataset}/Total-Score.pkl',weights_only=False)
    sample_IF = torch.load(f'Influence/{task}/{dataset}/score.pkl',weights_only=False)
    sample_IF = z_score_normalize(sample_IF)

    ppl_array = np.zeros(len(total_score))
    for process_num in range(1,9,1): ## maximum of k process
        if os.path.exists('ppl/{}/{}/ppl-init-{}.csv'.format(task,dataset,process_num)): ## i-th gradient 
            ppl_df = pd.read_csv('ppl/{}/{}/ppl-init-{}.csv'.format(task,dataset,process_num),index_col=0)
            for index,row in ppl_df.iterrows():
                ppl_array[index] = row[0]

    # QuRating_Score = np.load(f'QuRating/data/select_index_main_writing/{task}-{dataset}-QuRating-score.npy',allow_pickle=True).item()
    # for key in range(len(total_score)):
    #     # total_score[i] = total_score[i] - ppl_array[i]
    #     if sample_IF['iterative'].__contains__(key):
    #         total_score[key] += sample_IF['iterative'][key]
    #     # total_score[i] = total_score[i] + 6 * QuRating_Score[i]
    for key in sample_IF['iterative'].keys():
        total_score[key] += sample_IF['iterative'][key]


    p2_choose_index = []
    count = 0
    for i in cluster_rank.keys():
        index_all = cluster_rank[i]
        sorted_score_all = total_score[index_all]
        sorted_index_all = index_all[np.argsort(-sorted_score_all)]
        # print(index_all,sorted_index_all)
        # break
        change = cluster_per_budget - len(set(index_all[:cluster_per_budget]).intersection(set(sorted_index_all[:cluster_per_budget])))
        count += change
        p2_choose_index.extend(sorted_index_all[:cluster_per_budget])
    select_IF_df = train_file.iloc[p2_choose_index]
    print(len(select_IF_df) / len(train_file))
    json.dump(select_IF_df.to_dict(orient='records'), open('train/{}/{}/train-select-w-main-size-10.json'.format(task,dataset), 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

/tmp/ipykernel_100015/3535340534.py:44: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ppl_array[index] = row[0]
/tmp/ipykernel_100015/3535340534.py:44: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ppl_array[index] = row[0]
/tmp/ipykernel_100015/3535340534.py:44: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ppl_array[index] = row[0]
/tmp/ipykernel_100015/3535340534.py:44: FutureWarning: Series.__getitem__ treating keys as positions

0.09059286146400485


/tmp/ipykernel_100015/3535340534.py:44: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ppl_array[index] = row[0]


In [16]:
python Ablation_Study.py --yaml_path script/config_RE_RE.yaml --device 0,1 --model mistral-7B --DO_TRAIN_MAIN_VAR --DO_EVAL_MAIN_VAR --main_var IF-single-size-10 IF-single-size-20 DSIR-size-10 DSIR-size-20 SuperFiltering-size-10 SuperFiltering-size-20

0.2066997145861499

In [ ]:
python Ablation_Study.py --yaml_path script/config_ER_amazon-google.yaml --device 2,3 --model mistral-7B --DO_TRAIN_MAIN_VAR --DO_EVAL_MAIN_VAR --main_var IF-single-size-10 IF-single-size-20 DSIR-size-10 DSIR-size-20 SuperFiltering-size-10 SuperFiltering-size-20

### DataInf

In [ ]:
IF-single-size-10 IF-single-size-20 DSIR-size-10 DSIR-size-20 SuperFiltering-size-10 SuperFiltering-size-20

In [28]:
task = 'ER'
dataset = 'amazon-google'
select_num_df = pd.read_json('train/{}/{}/train-select-w-main-size-10.json'.format(task,dataset))
select_num = len(select_num_df)
yaml_path = f'script/config_{task}_{dataset}.yaml'
config = load_yaml(yaml_path)
train_file_path = config['train_file_path']
train_file = pd.read_json(train_file_path)
if os.path.exists('Influence_single/{}/{}'.format(task,dataset)):
    sample_IF_single = torch.load('Influence_single/{}/{}/score.pkl'.format(task,dataset),weights_only=False)
    for IF_method in sample_IF_single.keys():
        select_IF_single = get_top_k_indices(sample_IF_single[IF_method],k=select_num,IF=True)
        select_IF_single_df = train_file.iloc[select_IF_single]
        json.dump(select_IF_single_df.to_dict(orient='records'), open('train/{}/{}/train-select-w-IF-single-size-10.json'.format(task,dataset,IF_method), 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

## DSIR

### SuperFilter

In [32]:
task = 'ER'
dataset = 'amazon-google'
percentage = 10
select_num_df = pd.read_json(f'train/{task}/{dataset}/train-select-w-main-size-{percentage}.json')
select_size = len(select_num_df)
step_3_command = f'CUDA_VISIBLE_DEVICES=3 python code_ifd/select_data.py \
--json_data_path output/{task}-{dataset}-gpt2.jsonl \
--json_save_path ../DataSelection-IF/train/{task}/{dataset}/train-select-w-SuperFiltering-size-{percentage}.json \
--sample_num {select_size} '
step_3_command

'CUDA_VISIBLE_DEVICES=3 python code_ifd/select_data.py --json_data_path output/ER-amazon-google-gpt2.jsonl --json_save_path ../DataSelection-IF/train/ER/amazon-google/train-select-w-SuperFiltering-size-10.json --sample_num 599 '

## QuRating

In [33]:
np.load('eval_result/mistral-7B-RE-RE.npy',allow_pickle=True).item()

{'IF-single-iterative': {'acc': 0.8711389961389961,
  'micro_f1': 0.7471026490066226,
  'macro_f1': 0.5905163780263318},
 'IF-single-identity': {'acc': 0.8426640926640927,
  'micro_f1': 0.722682119205298,
  'macro_f1': 0.55191678272745},
 'IF-single-proposed': {'acc': 0.8537644787644788,
  'micro_f1': 0.732201986754967,
  'macro_f1': 0.5632134148611553},
 'main': {'acc': 0.9005791505791506,
  'micro_f1': 0.7723509933774834,
  'macro_f1': 0.5830344674616523},
 'ppl': {'acc': 0.8513513513513513,
  'micro_f1': 0.7301324503311257,
  'macro_f1': 0.5310203623037854},
 'FL': {'acc': 0.9073359073359073,
  'micro_f1': 0.7781456953642383,
  'macro_f1': 0.5600671023778724},
 'IF': {'acc': 0.8388030888030888,
  'micro_f1': 0.7193708609271523,
  'macro_f1': 0.5556746386823592},
 'QuRating': {'acc': 0.9145752895752896,
  'micro_f1': 0.7843543046357615,
  'macro_f1': 0.5714786595124909},
 'main-QuRating': {'acc': 0.8943050193050193,
  'micro_f1': 0.7669701986754967,
  'macro_f1': 0.576607674834364},
